In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import Dense,Dropout,Flatten,BatchNormalization
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from keras.applications.resnet_v2 import ResNet50V2
#from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data_dir="/content/drive/MyDrive/ctfolder/Fold3/train"
validation_data_dir="/content/drive/MyDrive/ctfolder/Fold3/test"
img_width=224
img_height=224
batch_size=32

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(directory=train_data_dir,
                                              target_size = (img_width, img_height),
                                              classes=['covid','normal'],
                                              class_mode = 'binary',
                                              batch_size=batch_size)

validation_generator = datagen.flow_from_directory(directory=validation_data_dir,
                                              target_size = (img_width, img_height),
                                              classes=['covid','normal'],
                                              class_mode = 'binary',
                                              batch_size=batch_size)

Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [ ]:
vgg_arch=ResNet50V2(input_shape=(img_width,img_height,3),weights="imagenet",include_top=False)

In [ ]:
for layers in vgg_arch.layers:
  layers.trainable=False

In [ ]:
model=Sequential()
model.add(vgg_arch)
model.add(Flatten())
model.add(Dense(128,activation='relu',))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(1,activation="sigmoid", name='visualized_layer'))
#model.add(Dense( 2, activation='sigmoid', name='visualized_layer'))




In [ ]:
model.summary()


In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
history=model.fit_generator(generator=train_generator, steps_per_epoch=len(train_generator), epochs = 5, 
                              validation_data=validation_generator, validation_steps=len(validation_generator),
                            verbose= 1
                              )

In [ ]:
pip install tf-keras-vis tensorflow

In [ ]:
import tensorflow as tf

from tensorflow.keras.applications.vgg16 import VGG16 as Model
from tensorflow.keras.applications.vgg16 import preprocess_input

import numpy as np
from matplotlib import pyplot as plt
# %matplotlib inline
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils import normalize
from vis.utils import utils
from tensorflow.keras.applications.vgg16 import decode_predictions
import json

In [ ]:
#utility function to show images
def display_imgs(images):
  subprot_args = {
   'nrows': 1,
   'ncols': 4,
   'figsize': (6, 3),
   'subplot_kw': {'xticks': [], 'yticks': []}
  }
  f, ax = plt.subplots(**subprot_args)
  for i in range(len(images)):
    ax[i].imshow(images[i])
  plt.tight_layout()
  plt.show()

In [ ]:
#Load images
from tensorflow.keras.preprocessing.image import load_img, img_to_array
img1 = load_img('/content/drive/MyDrive/ctfolder/Fold3/test/covid/covid400.png', target_size=(224, 224))
img2 = load_img('/content/drive/MyDrive/ctfolder/Fold3/test/covid/covid401.png', target_size=(224, 224))
img3 = load_img('/content/drive/MyDrive/ctfolder/Fold3/test/covid/covid402.png', target_size=(224, 224))
img4 = load_img('/content/drive/MyDrive/ctfolder/Fold3/test/covid/covid403.png', target_size=(224, 224))

In [ ]:
images = np.asarray([np.array(img1), np.array(img2), np.array(img3), np.array(img4)])
display_imgs(images)

In [ ]:
#convert to numpy array for reshaping
#convert to numpy array for reshaping
img1 = img_to_array(img1)
img2 = img_to_array(img2)
img3 = img_to_array(img3)
img4 = img_to_array(img4)

In [ ]:
#reshape to prepare for processing
img1 = img1.reshape(1,224,224,3)
img2 = img2.reshape(1,224,224,3)
img3 = img3.reshape(1,224,224,3)
img4 = img4.reshape(1,224,224,3)

In [ ]:
#preprocess to prepare for input
img1 = preprocess_input(img1)
img2 = preprocess_input(img2)
img3 = preprocess_input(img3)
img4 = preprocess_input(img4)

In [ ]:
# predictions with input images
yhat1 = model.predict(img1)
yhat2 = model.predict(img2)
yhat3 = model.predict(img3)
yhat4 = model.predict(img4)

In [ ]:
#decode predictions
label1 = decode_predictions(yhat1)
label2 = decode_predictions(yhat2)
label3 = decode_predictions(yhat3)
label4 = decode_predictions(yhat4)

In [ ]:
# extract top most prediction for each input
label1 = label1[0][0]
label2 = label2[0][0]
label3 = label3[0][0]
label4 = label4[0][0]

In [ ]:
# swap softmax layer with linear layer 
layer_idx = utils.find_layer_idx(model, 'visualized_layer')
model.layers[-1].activation = tf.keras.activations.linear
model = utils.apply_modifications(model)

In [ ]:
#get the input image index
from tf_keras_vis.utils.scores import CategoricalScore
#cat - 281, dog -235 , hen -8, tiger - 292
score = CategoricalScore([56, 35, 8 , 92])

In [ ]:
from matplotlib import cm
from tf_keras_vis.gradcam import Gradcam

input_classes = ['Covid19', 'Normal', 'Pneu_bac', 'Pneu_vir']

input_images = preprocess_input(images)

# Create Gradcam object
gradcam = Gradcam(model,
                  clone=True)

# Generate heatmap with GradCAM
cam = gradcam(score,
              input_images,
              penultimate_layer= -1)

In [ ]:
#show generated images
f, ax = plt.subplots(nrows=1, ncols=4, figsize=(12, 4))
for i, img_class in enumerate(input_classes):
    heatmap = np.uint8(cm.jet(cam[i])[..., :4] * 255)
    ax[i].set_title(img_class, fontsize=16)
    ax[i].imshow(images[i])
    ax[i].imshow(heatmap, cmap='jet', alpha=0.5) # overlay
    ax[i].axis('off')
plt.tight_layout()
plt.show()